In [1]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [2]:
pandf = fdr.DataReader("KS11","2009.01.01","2019-01-01")
pandf[75:]

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2009-04-21,1336.81,1307.95,1338.48,1303.22,639590000.0,0.0003
2009-04-22,1356.02,1347.79,1360.41,1336.41,730160000.0,0.0144
2009-04-23,1368.80,1367.30,1371.95,1353.16,747020000.0,0.0094
2009-04-24,1354.10,1371.81,1375.80,1343.55,683430000.0,-0.0107
2009-04-27,1339.83,1358.20,1366.66,1334.02,547300000.0,-0.0105
...,...,...,...,...,...,...
2018-12-21,2061.49,2052.70,2061.51,2049.76,311390000.0,0.0007
2018-12-24,2055.01,2050.38,2059.94,2046.18,285280000.0,-0.0031
2018-12-26,2028.01,2028.81,2037.83,2014.28,321500000.0,-0.0131


In [3]:
# convert nparray
nparr = pandf.values
nparr.astype('float32')
print(nparr)

[[ 1.15740e+03  1.13287e+03  1.15740e+03  1.11884e+03  3.40780e+08
   2.93000e-02]
 [ 1.17357e+03  1.18098e+03  1.18522e+03  1.16666e+03  4.14760e+08
   1.40000e-02]
 [ 1.19428e+03  1.18352e+03  1.20359e+03  1.17994e+03  4.44200e+08
   1.76000e-02]
 ...
 [ 2.02801e+03  2.02881e+03  2.03783e+03  2.01428e+03  3.21500e+08
  -1.31000e-02]
 [ 2.02844e+03  2.03209e+03  2.03557e+03  2.02139e+03  3.98020e+08
   2.00000e-04]
 [ 2.04104e+03  2.03670e+03  2.04697e+03  2.03541e+03  3.52680e+08
   6.20000e-03]]


In [36]:
# normalization
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)

In [37]:
nptf=nptf[:-1]

In [52]:
# split train, test
y=np.array(pandf["Change"].shift(-1).dropna()).reshape(-1,1)
train_size = int(len(nptf) * 0.7)
y_train=y[:train_size]
y_test=y[train_size:]
X_train = nptf[:train_size]
X_test = nptf[train_size:]

In [53]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [62]:
# simple lstm network learning
model = Sequential()
model.add(SimpleRNN(36, input_shape=(1, 6)))
for i in range(5):
    model.add(Dense(36,activation='sigmoid'))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam', metrics=['MSE'])
model.fit(X_train, y_train, epochs=100, batch_size=16,validation_split=0.1)
model.evaluate(X_test,y_test)

Train on 1557 samples, validate on 174 samples
Epoch 1/100
1557/1557 [==============================] - 1s 579us/step - loss: 7.6040e-04 - MSE: 7.6040e-04 - val_loss: 7.8204e-05 - val_MSE: 7.8204e-05
Epoch 2/100
1557/1557 [==============================] - 0s 214us/step - loss: 1.4405e-04 - MSE: 1.4405e-04 - val_loss: 1.2422e-04 - val_MSE: 1.2422e-04
Epoch 3/100
1557/1557 [==============================] - 0s 184us/step - loss: 1.3762e-04 - MSE: 1.3762e-04 - val_loss: 1.4539e-04 - val_MSE: 1.4539e-04
Epoch 4/100
1557/1557 [==============================] - 0s 176us/step - loss: 1.6512e-04 - MSE: 1.6512e-04 - val_loss: 8.8980e-05 - val_MSE: 8.8980e-05
Epoch 5/100
1557/1557 [==============================] - 0s 136us/step - loss: 1.4023e-04 - MSE: 1.4023e-04 - val_loss: 8.4526e-05 - val_MSE: 8.4526e-05
Epoch 6/100
1557/1557 [==============================] - 0s 156us/step - loss: 1.9966e-04 - MSE: 1.9966e-04 - val_loss: 8.7518e-05 - val_MSE: 8.7518e-05
Epoch 7/100
1557/1557 [============

Epoch 54/100
1557/1557 [==============================] - 0s 193us/step - loss: 1.4903e-04 - MSE: 1.4903e-04 - val_loss: 1.4635e-04 - val_MSE: 1.4635e-04
Epoch 55/100
1557/1557 [==============================] - 0s 174us/step - loss: 1.5976e-04 - MSE: 1.5976e-04 - val_loss: 9.3681e-05 - val_MSE: 9.3681e-05
Epoch 56/100
1557/1557 [==============================] - 0s 151us/step - loss: 1.5632e-04 - MSE: 1.5632e-04 - val_loss: 1.0348e-04 - val_MSE: 1.0348e-04
Epoch 57/100
1557/1557 [==============================] - 0s 140us/step - loss: 1.5004e-04 - MSE: 1.5004e-04 - val_loss: 9.1271e-05 - val_MSE: 9.1271e-05
Epoch 58/100
1557/1557 [==============================] - 0s 150us/step - loss: 1.4909e-04 - MSE: 1.4909e-04 - val_loss: 1.2295e-04 - val_MSE: 1.2295e-04
Epoch 59/100
1557/1557 [==============================] - 0s 145us/step - loss: 1.6602e-04 - MSE: 1.6602e-04 - val_loss: 7.9479e-05 - val_MSE: 7.9479e-05
Epoch 60/100
1557/1557 [==============================] - 0s 153us/step - lo

[6.447593796478316e-05, 6.447594205383211e-05]

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(np.where(y_test>0,1,0),np.where(model.predict(X_test)>0,1,0))

0.4716981132075472

In [13]:
a=model.evaluate(X_test,y_test)

742/742 [==============================] - 0s 42us/step


In [58]:
y_test

array([[ 0.0043],
       [-0.0013],
       [ 0.003 ],
       [ 0.0057],
       [ 0.0033],
       [-0.0043],
       [-0.0134],
       [ 0.0011],
       [-0.0025],
       [-0.0217],
       [ 0.0061],
       [-0.0026],
       [-0.011 ],
       [ 0.007 ],
       [-0.0119],
       [-0.0021],
       [ 0.0134],
       [-0.0085],
       [-0.0111],
       [-0.0002],
       [ 0.006 ],
       [-0.0234],
       [-0.0027],
       [ 0.0211],
       [ 0.0074],
       [-0.0115],
       [ 0.014 ],
       [ 0.0048],
       [ 0.0027],
       [ 0.0067],
       [-0.0095],
       [-0.0084],
       [ 0.0135],
       [ 0.0008],
       [-0.0293],
       [-0.0141],
       [ 0.0147],
       [ 0.014 ],
       [-0.0023],
       [ 0.0132],
       [ 0.0039],
       [ 0.0001],
       [-0.0011],
       [-0.0009],
       [ 0.0032],
       [ 0.0008],
       [-0.0018],
       [ 0.016 ],
       [ 0.0055],
       [-0.0013],
       [ 0.0011],
       [-0.006 ],
       [ 0.0035],
       [ 0.0084],
       [ 0.0011],
       [ 0

In [51]:
y[:20]

array([[0.62819456],
       [0.65787304],
       [0.74690849],
       [0.36191261],
       [0.34377576],
       [0.34377576],
       [0.59109646],
       [0.61830173],
       [0.01566364],
       [0.69002473],
       [0.62489695],
       [0.34212696],
       [0.34295136],
       [0.6067601 ],
       [0.34377576],
       [1.        ],
       [0.57378401],
       [0.48145095],
       [0.40560594],
       [0.62984336]])

In [4]:
X_train

NameError: name 'X_train' is not defined

In [1]:
a=[1,2,3,4,5]
a=a[:-1]

In [2]:
a

[1, 2, 3, 4]

In [3]:
nptf

NameError: name 'nptf' is not defined

In [ ]:
for x in feature_df:
    print(x)